# <center>Big Data for Engineers &ndash; Exercises</center>
## <center>Spring 2022 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and SparkSQL</center>

# Preparation for the exercise in Spark

1. Change to `exercise09` repository

2. Start docker <br>
```docker-compose up -d``` <br>
(This process can take up to 10 minutes.)

3. After docker finishes downloading the images, you should be able to start the jupyter notebook by copying the following URL to your browser <br>
```http://localhost:8888/lab```

## <center>1. Spark Dataframes</center>

Spark Dataframes allow the user to perform simple and efficient operations on data, as long as the data is structured and has a schema. Dataframes are similar to relational tables in relational databases: conceptually a dataframe is a specialization of a Spark RDD with schema information attached. You can find more information in Karau, H. et al. (2015). Learning Spark, Chapter 9 (optional reading).

Throughout the exercise, you can see the equivalency of Spark RDD, Spark Dataframes and SparkSQL. 

### 1.1. Data preprocessing

In [1]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "orders.jsonl"
orders_df = spark.read.json(path).cache()

The type of our dataset object is DataFrame.

In [2]:
type(orders_df)

pyspark.sql.dataframe.DataFrame

Print the schema.

In [3]:
orders_df.printSchema()

root
 |-- customer: struct (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |-- date: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- price: double (nullable = true)
 |    |    |-- product: string (nullable = true)
 |    |    |-- quantity: long (nullable = true)
 |-- order_id: long (nullable = true)



Print one row.

In [4]:
orders_df.limit(1).collect()

[Row(customer=Row(first_name='Preston', last_name='Landry'), date='2018-2-4', items=[Row(price=1.53, product='fan', quantity=5), Row(price=1.33, product='computer screen', quantity=6), Row(price=1.06, product='kettle', quantity=6), Row(price=1.96, product='stuffed animal', quantity=3), Row(price=1.09, product='the book', quantity=7), Row(price=1.42, product='headphones', quantity=9), Row(price=1.67, product='whiskey bottle', quantity=3)], order_id=0)]

You can access the underlying RDD object and use any functions you learned for Spark RDDs.

In [ ]:
orders_df.rdd.filter(lambda ordr: ordr.customer.last_name == "Landry").count()

### 1.2. Dataframe Operations
We perform some queries using operations on Dataframes ([Here](https://spark.apache.org/docs/2.3.0/sql-programming-guide.html#untyped-dataset-operations-aka-dataframe-operations) is a guide on DF Operations with a link to the [API Documentation](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html))

We can select columns and show the results.

In [ ]:
orders_df.select("customer.first_name", "customer.last_name").limit(5).show()

As you can see we can navigate to the nested items with the dot.

In [ ]:
orders_df.filter(orders_df["customer.last_name"] == "Landry").count()

How about nested arrays?

In [ ]:
orders_df.select("order_id", "items").orderBy("order_id").limit(5).show()

Let us try to find orders of a fan.

In [ ]:
orders_df.filter(orders_df["items.product"] == "fan").count()

The above code doesn't work! Use [```array_contains```](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.array_contains.html) instead.

In [ ]:
from pyspark.sql.functions import array_contains

orders_df.filter(array_contains("items.product", "fan")).count()

<b>Let us try to unnest the data.</b>

Unnest the products with [`explode`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.explode.html?highlight=explode#pyspark.sql.functions.explode).

`explode` will generate as many rows as there are elements in the array and match them to other attributes via projection.

In [ ]:
from pyspark.sql.functions import explode

orders_df.select(explode("items").alias("i"), "i.product", "order_id").orderBy("order_id").limit(5).show()

Now we can use this table to filter.

In [ ]:
exploded_df = orders_df.select(explode("items").alias("i"), "i.product", "order_id")
exploded_df.filter(exploded_df["product"] == "fan").count()

You might have tried to access the `i.product` column directly using a ```.filter``` right after the ```.select```. That, however, does not work, because the column is not available to ```orders_df``` when creating a clause like ```(orders_df["i.product"] == "fan")```. A possible workaround when using Dataframe operations is to use a string clause in ```.filter```, so that the product column will be resolved after it has been added with the ```.select```.

In [ ]:
orders_df.select(explode("items").alias("i"), "i.product", "order_id").filter("product = 'fan'").count()

Any ideas why there are more "fan" in the `explode` query than the `array_contain` one? 

This is because that there could be more than one "fan" types in each order, and we would therefore count some orders with the same `order_id` more than one time. You will find about that when inspecting the `orders.jsonl` data. 
E.g., 
```json
{"order_id": 2, "date": "2016-6-6", "customer": {"first_name": "Brendon", "last_name": "Sicilia"}, "items": [..., {"product": "fan", "quantity": 7, "price": 1.1}, ..., {"product": "fan", "quantity": 8, "price": 1.15}]}
```

<b>Project the nested columns.</b>

In [ ]:
orders_df.select(explode("items").alias("i"), "*").select(
    "order_id", "customer.*", "date", "i.*").limit(3).show()

### 1.3. Exercises

1) Find the average quantity at which each product is purchased. Only show the top 10 products by average quantity. <br> 
(Hint: You may need to import the function [`desc`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.desc.html?highlight=desc) from ```pyspark.sql.functions``` to define descending order)

In [ ]:
from pyspark.sql.functions import desc

# your code here

2) Find the most expensive order. <br>
(Hint: You first build a dataframe by `explode`'ing the items. Then you calculate the total price and aggregate per order. 

In [ ]:
exploded_df = ...

## <center>2. Spark SQL</center>

Spark SQL allows the users to formulate their queries using SQL. The requirement is the use of Dataframes, which as said before are similar to relational tables. In addition to a familiar interface, writing queries in SQL might provide better performance than RDDs, inheriting efficiency from the Dataframe operations, while also performing automatic optimization of queries.

First we need to install the `sparksql` magic command.

In [ ]:
!pip install sparksql-magic

In [ ]:
%load_ext sparksql_magic

In order to use SQL we need to create a temporary table.

<b>Note this table only exists for the current session.</b>

In [ ]:
orders_df.createOrReplaceTempView("orders")

### 2.1. Queries

Finally, run SQL queries on the registered table `orders`. We will run the same queries as during the previous section, but with SQL.

As you can see we can navigate to the nested items with the dot.

In [ ]:
%%sparksql
-- Finally, run SQL queries on the registered table "orders"
-- As you can see we can navigate to the nested items with the dot
SELECT count(*)
FROM orders
WHERE orders.customer.last_name == "Landry"

How about nested arrays?

In [ ]:
%%sparksql
-- How about nested arrays?
SELECT order_id, items
FROM orders AS o
ORDER BY order_id
LIMIT 5

Let us try to find orders of a fan.

In [ ]:
%%sparksql 
SELECT count(*)
FROM orders
WHERE items.product = "fan"

The above code doesn't work! Use [```array_contains```](https://spark.apache.org/docs/latest/api/sql/index.html#array_contains) instead.

In [ ]:
%%sparksql

SELECT count(*)
FROM orders
WHERE array_contains(items.product, "fan")

Let us try to unnest the data.

Unnest the products with [`explode`](https://spark.apache.org/docs/latest/api/sql/index.html#explode).

`explode` will generate as many rows as there are elements in the array and match them to other attributes.

In [ ]:
%%sparksql
SELECT explode(items) as i, i.product, order_id
FROM orders
ORDER BY order_id
limit 5

Now we can use this table to filter.

In [ ]:
%%sparksql
-- Filter on product
SELECT count(*)
    FROM (
    SELECT explode(items) as i, i.product, order_id
    FROM orders
    ORDER BY order_id
    )
WHERE product = "fan"

You might have tried to access the `i.product` column directly in the same ```SELECT``` clause. That, however, does not work, because the column is not available to the ```WHERE``` clause. In order to access the built columns directly, we need to unnest the data and make it part of our ```FROM``` clause. [```LATERAL VIEW```](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-lateral-view.html) lets us do just that, matching each non-array attribute to an unnested row from the array.  

In [ ]:
%%sparksql
SELECT *
FROM orders LATERAL VIEW explode(items) as flat_items
WHERE flat_items.product = "fan"
ORDER BY order_id
LIMIT 4

Project the nested columns.

In [ ]:
%%sparksql
SELECT order_id, customer.first_name, customer.last_name, date, flat_items.*
FROM orders LATERAL VIEW explode(items) item_table as flat_items
WHERE flat_items.product = "fan"
ORDER BY order_id
LIMIT 3

Having built an unnested table, we can now easily aggregate over the previously nested columns.

### 2.2. Exercises

1) Find the average quantity at which each product is purchased. Only show the top 10 products by quantity. 

In [ ]:
%%sparksql
-- your code here

2) Find the most expensive order.

In [ ]:
%%sparksql
-- your code here

## <center>3. Create Nestedness (Optional)</center>

We've already had a look at the solution of dataframes/SparkSQL towards <b>unnesting</b> arrays by using `explode` method. For the other way round, Spark Dataframes / Spark SQL also provide ways for us to nest our data by creating arrays, especially after clauses like `groupBy`.

In traditional PostgreSQL, we have to use one of the aggregation functions (`max, sum, count,`...) to process the result after the `groupBy` operation. For example, for each customer (assume there are no customers with both the same first name and last name), we want to find out the number of distinct dates when they placed an order. You can fill in the queries in both Spark DataFrames and Spark SQL. The query could look like this using [`countDistinct`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.countDistinct.html?highlight=countdistinct#pyspark.sql.functions.countDistinct):

In [ ]:
from pyspark.sql.functions import countDistinct
orders_df.groupBy("customer.first_name", "customer.last_name").agg(countDistinct("date")).show()

In [ ]:
%%sparksql
SELECT customer.first_name, customer.last_name, count(DISTINCT date) 
FROM orders 
GROUP BY customer.first_name, customer.last_name

But what if we are interested not only in the count of distinct dates, but the actual
dates themselves? Luckily Spark Dataframes / Spark SQL do provide us with methods to preserve the original information of the date list. If now we would like to know for each customer, on which dates they placed an order, we shall use [`collect_set`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.collect_set.html?highlight=collect_set#pyspark.sql.functions.collect_set) method:

In [ ]:
from pyspark.sql.functions import collect_set
orders_df.groupBy("customer.first_name", "customer.last_name").agg(collect_set("date")).show()

In [ ]:
%%sparksql
SELECT customer.first_name, customer.last_name, collect_set(date)
FROM orders 
GROUP BY customer.first_name, customer.last_name

For some other cases where we want to preserve all the entries rather than the de-duplicated ones, we can use  [`collect_list`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.collect_list.html?highlight=collect_list#pyspark.sql.functions.collect_list) method. For example, for each date we want to record the last names of customers. Since two customers might share the same last name, we need to collect all of them. The query should look like this:

In [ ]:
from pyspark.sql.functions import collect_list
orders_df.groupBy("date").agg(collect_list("customer.last_name")).show()

In [ ]:
%%sparksql
SELECT date, collect_list(customer.last_name) 
FROM orders 
GROUP BY date
LIMIT 2

Now try it on yourself.

For every order on 2016-1-1, return the list of products that appeared in that order:

In [ ]:
from pyspark.sql.functions import explode
exploded_df = ...

For every product, return the set of dates when it's purchased:

In [ ]:
from pyspark.sql.functions import collect_set
# your code here

One of the drawbacks of the <font face="courier">collect_set/collect_list</font> method is they only accept one column as the argument. Later we will see how we can create nestedness on pretty much everything after we get the hang of the mighty JSONiq.